In [83]:
import requests
import json
import os
from tqdm.notebook import tqdm

In [84]:
def get_pfams_ids(path):
    data = []
    with open(pfam_family_file) as f:
        data = f.readlines()
    return [d.strip() for d in data]

pfam_family_file = os.path.join("dataset", "relevant_pfam_ids.txt")

pfam_ids = get_pfams_ids(pfam_family_file)

In [85]:
def get_description(id):
    response = requests.get(f"https://www.ebi.ac.uk/interpro/api/entry/pfam/{id}")
    return response.json()

def get_description_mock(id):
    return {"metadata" : {"description" : "mock"}}

def collate_all_families(pfam_familiy_ids, mock=True):
    dataset = {}
    for pfam_id in tqdm(pfam_familiy_ids):
        if mock:
            dataset[pfam_id] = get_description_mock(pfam_id)
        else:
            dataset[pfam_id] = get_description(pfam_id)    

    return dataset


In [86]:
with open(os.path.join("dataset", "pfam_data.json"), "w") as f:
    json.dump(collate_all_families(pfam_ids, mock=False), f)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [87]:
with open(os.path.join("dataset", "pfam_data.json")) as f:
    data = json.load(f)

In [98]:
hist = []
with open(os.path.join("dataset", "pfam_to_id.json")) as f:
    pfam_id_to_number = json.load(f)

BERT_model_idx2label = {}
for family_id in tqdm(pfam_ids):
    name = data[family_id]["metadata"]["name"]["name"]
    description = data[family_id]["metadata"]["description"]

    # print(f"{family_id} -> {name}, {description}")
    entry = f"{name}"
    if description is not None:
        desc = description[0].replace("<p>", "").replace("</p>", "")
        entry = f"{name}\n{desc}"
    BERT_model_idx2label[pfam_id_to_number[family_id]] = entry

  0%|          | 0/1000 [00:00<?, ?it/s]

In [101]:
with open(os.path.join("dataset", "BERT_model_idx2label.json"), "w") as f:
    json.dump(BERT_model_idx2label, f)

In [100]:
BERT_model_idx2label

{0: 'HipA-like C-terminal domain\nThe members of this family are similar to a region close to the C-terminus of the HipA protein expressed by various bacterial species (for example Swiss:P23874). This protein is known to be involved in high-frequency persistence to the lethal effects of inhibition of either DNA or peptidoglycan synthesis [PMID:1715862]. When expressed alone, it is toxic to bacterial  cells [PMID:1715862], but it is usually tightly associated with  HipB [PMID:8021189], and the HipA-HipB complex may be involved in autoregulation of the hip operon. The hip proteins may be involved in cell division control and may interact with  cell division genes or their products [PMID:8021189].',
 1: 'CheB methylesterase',
 2: 'AMMECR1\nThis family consists of several AMMECR1 as well as several uncharacterised proteins. The contiguous gene deletion syndrome AMME is characterised by Alport syndrome, midface hypoplasia, mental retardation and elliptocytosis and is caused by a deletion in